In [12]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import csv

In [2]:
dfx = pd.read_csv('Train.csv')
dfx.head(n=10)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

dfx["label"] = le.fit_transform(dfx["label"])

dfx.head(n=5)

,review,label
0,mature intelligent and highly charged melodram...,1
1,http://video.google.com/videoplay?docid=211772...,1
2,Title: Opera (1987) Director: Dario Argento Ca...,1
3,I think a lot of people just wrote this off as...,1
4,This is a story of two dogs and a cat looking ...,1


In [3]:
x = dfx.iloc[:,0]
y = dfx.iloc[:,1]

x = np.array(x)
y = np.array(y)

tokenizer = RegexpTokenizer(r'\w+')
en_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

In [4]:
def getStemmedReview(review):
    
    review = review.lower()
    review = review.replace("<br /><br />"," ")
    
    
    tokens = tokenizer.tokenize(review)
    new_tokens = [token for token in tokens if token not in en_stopwords]
    stemmed_tokens = [ps.stem(token) for token in new_tokens]
    
    cleaned_review = ' '.join(stemmed_tokens)
    
    return cleaned_review

In [5]:
x_test = pd.read_csv('Test.csv').values
x_test = x_test.reshape(-1,)

x_clean = [getStemmedReview(i) for i in x]
x_cleant = [getStemmedReview(i) for i in x_test]

In [6]:
print(x_test.shape)

(10000,)


In [7]:
cv = CountVectorizer(ngram_range=(1,2))

x_vec = cv.fit_transform(x_clean)

print(x_vec.shape)

(40000, 2270363)


In [8]:
xt_vec = cv.transform(x_cleant)
print(xt_vec.shape)

(10000, 2270363)


In [9]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

mnb.fit(x_vec,y)

Yx = mnb.predict(xt_vec) 

print(Yx)

[0 0 0 ... 1 1 0]


In [10]:
mnb.score(x_vec,y)

0.9946

In [14]:
answer = le.inverse_transform(Yx)

In [15]:
print(answer)


['neg' 'neg' 'neg' ... 'pos' 'pos' 'neg']


In [21]:
with open('Samples_Submission.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    m = answer.shape[0]
    writer.writerow(["Id","label"])
    for i in range(m):
        l = [i,answer[i]]
        writer.writerow(l)
    csvFile.close()